In [2]:
import os

os.chdir('../')

In [ ]:
remote_server_uri = "http://ec2-18-188-232-57.us-east-2.compute.amazonaws.com:5000/"
mlflow.set_tracking_uri(remote_server_uri)

In [2]:
os.environ["MLFLOW_TRACKING_URI"] = "http://ec2-18-222-222-1.us-east-2.compute.amazonaws.com:5000/"

In [8]:
import tensorflow as tf

model = tf.keras.models.load_model("artifacts/training/model.h5")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [4]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories, save_json


class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config  = EvaluationConfig(
            path_of_model = 'artifacts/training/model.h5',
            training_data = 'artifacts/data_ingestion/Chest-CT-Scan-data',
            mlflow_uri = 'http://ec2-18-222-222-1.us-east-2.compute.amazonaws.com:5000/',
            all_params=self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config

In [5]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [6]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
    
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model,"model", registered_model_name="VGG16_Model")
            
            else:
                mlflow.keras.log_model(self.model, "model")

    

In [9]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-07-07 16:01:50,884: INFO: 5(module)s: yaml file: config\config.yaml loaded successfully]
[2024-07-07 16:01:50,887: INFO: 5(module)s: yaml file: params.yaml loaded successfully]
[2024-07-07 16:01:50,888: INFO: 5(module)s: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 9s 1s/step - loss: 9.2378 - accuracy: 0.4314


2024/07/07 16:02:00 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-07-07 16:02:01,273: WARNING: 5(module)s: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2024-07-07 16:02:01,923: INFO: 5(module)s: Assets written to: C:\Users\saish\AppData\Local\Temp\tmpv3tt72pm\model\data\model\assets]
[2024-07-07 16:02:12,955: INFO: 5(module)s: Found credentials in shared credentials file: ~/.aws/credentials]


c:\Users\saish\OneDrive\Desktop\Projects\.venv\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16_Model'.
2024/07/07 16:02:35 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16_Model, version 1
Created version '1' of model 'VGG16_Model'.
